In [1]:
# imports
import pandas as pd
import numpy as np
from IPython.core.debugger import set_trace
import matplotlib.pyplot as plt
import subprocess

# Export notebook as python script to the ../python-code - folder

In [ ]:
subprocess.run("jupyter nbcbonvert --output-dir='../python-code' --to python Naive_Bayes.ipynb --TemplateExporter.exclude_markdown=True --TemplateExporter.exclude_input_prompt=True", shell=True)

In [ ]:
class Naive_Bayes_CF():
    """
    Class representing a Naive - Bayes classifier implementation for the collaborative filterting setting of recommender systems.
    """
    def __init__(self, rmh, is_task_conviction:bool=True):
        super().__init__()
        self.rmh_ = rmh
        if is_task_conviction:
            self.possibles_classes = set([0,1])
        else:
            self.possibles_classes = set([i for i in range(7)])
    
    def build_lookups(self) -> None:
        """
        Map users and items to numerical values for further indexing.
        """
        self.userid_lookup_ = {username: i for i, username in enumerate(self.rmh_.final_rating_matrix_w_usernames["username"])}
        self.itemid_lookup_ = {item: i for i, item in enumerate(list(self.rmh_.final_rating_matrix_w_usernames.columns))}
        # Reverse the two calculated mappings for bidirectional lookup
        self.username_lookup = {user_id: username for username, user_id in self.userid_lookup_.items()}
        self.itemname_lookup = {item_id: itemname for itemname, item_id in self.itemid_lookup_.items()}
    